In [19]:
# full pipeline 
%run -i data_preparation/v2.py
%run -i model_configuration/v2.py
%run -i model_training/v4.py


In [20]:
%load_ext tensorboard 


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## lunch tensorbard 

In [21]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 5088), started 1 day, 5:55:19 ago. (Use '!kill 5088' to kill it.)

In [22]:
from torch.utils.tensorboard import SummaryWriter


In [23]:
pwd

'd:\\ujjwal\\Deep-learning\\fundamentas'

In [24]:
writer = SummaryWriter('runs/test')

In [25]:
model.state_dict()

OrderedDict([('0.weight', tensor([[2.0481]], device='cuda:0')),
             ('0.bias', tensor([9.7007], device='cuda:0'))])

In [26]:
dummy_x , dummy_y = next(iter(train_loader))
dummy_x = dummy_x.view(-1,1)
dumym_y  = dummy_y.view(-1 , 1)

In [27]:
writer.add_graph(model , dummy_x.to(device))

In [28]:
writer.add_scalars(
  main_tag='loss',
  tag_scalar_dict={'training': loss,
                    'validation': val_loss},
  global_step= epcoh)

  

In [29]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 5088), started 1 day, 5:55:27 ago. (Use '!kill 5088' to kill it.)

In [30]:
%run -i data_preparation/v2.py

In [13]:
# %load model_configuration/v2.py
import torch 
import torch.optim as optim 
import torch.nn as nn 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.01 #set learning rate 
model = nn.Sequential(nn.Linear(1,1).to(device))
optimizer = optim.SGD(model.parameters() , lr = lr)
loss_fn = nn.MSELoss(reduction= 'mean')


def make_train_step_fn(model , loss_fn , optimizer):
    def train_step(x ,y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat , y)
        loss.backward() #calcualte gradients 
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

train_step_fn = make_train_step_fn(model , loss_fn , optimizer)

def make_val_step_fn(model , loss_fn):
    def perform_val_step(x ,y ):
        model.eval() #model evaluation 
        yhat = model(x)
        loss = loss_fn(yhat , y)
        return loss.item()
    return perform_val_step


val_step_fn = make_val_step_fn(model , loss_fn)


In [44]:
%%writefile model_configuration/v3.py
import torch 
import torch.optim as optim 
import torch.nn as nn 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.optim as optim

def make_train_step_fn(model , optimizer , loss_fn):
    def train_step(x ,y ):
        model.train() 
        yhat = model(x)
        loss = loss_fn(yhat , y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    
    return train_step 

def make_val_step_fn(model , loss_fn):
    def perform_val_step(x ,y ):
        model.eval()
        yhat = model(x)
        loss = loss_fn(yhat , y)
        return loss.item()
    return perform_val_step
        

#set lr rate 
lr = 0.01
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1 ,1 ).to(device))
optimizer = optim.SGD(model.parameters() , lr = lr)
loss_fn = nn.MSELoss(reduction = 'mean')

train_step_fn = make_train_step_fn(model , loss_fn , optimizer)
val_step_fn = make_val_step_fn(model , loss_fn)

#create a summary writter to interface with tensorboard 
writer = SummaryWriter('runs/simple_linear_regression')
#fetch single mini batch so we can use add_graph
x_dummy , y_dummy = next(iter(train_loader))
x_dummy = x_dummy.view(-1 , 1)
y_dummy = y_dummy.view(-1,1)
writer.add_graph(model , x_dummy.to(device))

Overwriting model_configuration/v3.py


In [45]:
%run -i model_configuration/v3.py

In [46]:
next(iter(train_loader))

[tensor([[ 5.0000],
         [ 2.0000],
         [ 4.9091],
         [ 4.0000],
         [ 4.0909],
         [ 4.6364],
         [ 8.0909],
         [ 1.4545],
         [ 9.5455],
         [ 6.2727],
         [10.0000],
         [ 6.4545],
         [ 5.5455],
         [ 8.9091],
         [ 5.1818],
         [ 1.2727]]),
 tensor([[20.0000],
         [14.0000],
         [19.8182],
         [18.0000],
         [18.1818],
         [19.2727],
         [26.1818],
         [12.9091],
         [29.0909],
         [22.5455],
         [30.0000],
         [22.9091],
         [21.0909],
         [27.8182],
         [20.3636],
         [12.5455]])]

In [47]:
next(iter(train_loader))

[tensor([[4.8182],
         [4.2727],
         [7.2727],
         [4.6364],
         [1.8182],
         [6.5455],
         [9.6364],
         [8.8182],
         [7.0909],
         [6.9091],
         [1.2727],
         [9.5455],
         [4.3636],
         [9.0000],
         [5.6364],
         [9.7273]]),
 tensor([[19.6364],
         [18.5455],
         [24.5455],
         [19.2727],
         [13.6364],
         [23.0909],
         [29.2727],
         [27.6364],
         [24.1818],
         [23.8182],
         [12.5455],
         [29.0909],
         [18.7273],
         [28.0000],
         [21.2727],
         [29.4545]])]

%run -i data_preparation/v2.py 
%run -i model_configuration/v3.py
%run -i model_training/v3.py


In [56]:
%%writefile model_training/v5.py
import numpy  as np 
def mini_batch(device , train_loader , train_step_fn):
    mini_batch_losses = []
    for x_batch , y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = train_step_fn(x_batch , y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_loss)
    return loss 

        

n_epcohs = 200 
losses = []
val_losses = [] 
for epcoh in range(n_epcohs):
    #inner loop 
    loss = mini_batch(device ,  train_loader , train_step_fn)
    losses.append(loss)

    with torch.no_grad():
        val_loss = mini_batch(device , val_loader , val_step_fn)
        val_losses.append(val_loss)

    writer.add_scalars(main_tag = 'loss' , 
                      tag_scalar_dict  = {'tranning': loss , 
                                         'validation': val_loss} , 
                      global_step = epcoh)
writer.close()

Writing model_training/v5.py


In [57]:
%run -i model_training/v5.py

In [58]:
model.state_dict()

OrderedDict([('0.weight', tensor([[2.0000]], device='cuda:0')),
             ('0.bias', tensor([9.9998], device='cuda:0'))])

In [63]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 5088), started 1 day, 6:31:08 ago. (Use '!kill 5088' to kill it.)

In [ ]:
%tensorboard --logdir runs/test

## save the checkpoint 
Training a model successfully is great, no doubt about that, but not all models will
train quickly, and training may get interrupted (computer crashing, timeout after
12 hours of continuous GPU usage on Google Colab, etc.). It would be a pity to have
to start over, right?
So, it is important to be able to checkpoint or save our model, that is, save it to disk


In [64]:
model.state_dict()

OrderedDict([('0.weight', tensor([[2.0000]], device='cuda:0')),
             ('0.bias', tensor([9.9998], device='cuda:0'))])

## model state

In [66]:
#fror checkpoint we basically have to state to a file so that it can be loaded later 
#save 
checkpoint = {'epoch': n_epochs,
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'loss': losses,
  'val_loss': val_losses}
torch.save(checkpoint, 'model_checkpoint.pth')


In [23]:
%tensorboard logdirs --runs

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] {serve} ...
tensorboard: error: argument {serve}: invalid choice: 'logdirs' (choose from 'serve')

In [27]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter('run/test22')

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('mnist_train', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
model = torchvision.models.resnet50(False)
# Have ResNet model take in grayscale rather than RGB
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
images, labels = next(iter(trainloader))

grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)
writer.add_graph(model, images)
writer.close()

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
            

In [29]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
%tensorboard --logdirs run

## ended 

In [ ]:
writer.add_graph(model , dummy_x.tp(device))


In [30]:
writer = SummaryWriter('runs/test')

In [31]:
writer.add_graph(model , dummy_x.to(device))

Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [16, 1]
Error occurs, No graph saved


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [16, 1]